In [1]:
import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
D=pd.read_csv('../data/Semenov2017PaperData_matches_dota.csv',nrows=2)
D=D.loc[D['leavers']==0,:]

In [3]:
Out=sparse.csc_matrix(D['radiant_win'])
train_heroes=D.loc[:,'r1_hero_id':]
trainX=np.zeros((D.shape[0],2*113))
for i in range(D.shape[0]):
    trainX[i,train_heroes.iloc[i,0:5].values-1]=np.ones(5)
    trainX[i,train_heroes.iloc[i,5:10].values-1+113]=np.ones(5)
In=sparse.csr_matrix(trainX)

In [4]:
m=10000
j=0
for chunk in pd.read_csv('../data/Semenov2017PaperData_matches_dota.csv',chunksize=m):
    if j%50==0:
        print('Processing ',j+1,'st block of 10000 records')
    j+=1
        
    D=chunk.loc[chunk['leavers']==0,:]
    n=D.shape[0]
    Out=sparse.hstack([Out,sparse.csc_matrix(D['radiant_win'])])
    train_heroes=D.loc[:,'r1_hero_id':]
    trainX=np.zeros((n,2*113))
    for i in range(D.shape[0]):
        trainX[i,train_heroes.iloc[i,0:5].values-1]=np.ones(5)
        trainX[i,train_heroes.iloc[i,5:10].values-1+113]=np.ones(5)
    In=sparse.vstack([In,sparse.csr_matrix(trainX)])

Processing  1 st block of 10000 records
Processing  51 st block of 10000 records
Processing  101 st block of 10000 records
Processing  151 st block of 10000 records
Processing  201 st block of 10000 records
Processing  251 st block of 10000 records
Processing  301 st block of 10000 records
Processing  351 st block of 10000 records
Processing  401 st block of 10000 records
Processing  451 st block of 10000 records
Processing  501 st block of 10000 records


In [5]:
print('Input Matrix size', In.shape)
print('Output Matrix size', Out.shape)

Input Matrix size (4182501, 226)
Output Matrix size (1, 4182501)


In [6]:
from sklearn.model_selection import train_test_split
tempX, TestX, tempY, TestY = train_test_split(In,Out.transpose(),test_size=0.25)
TrainX, ValidX, TrainY, ValidY = train_test_split(tempX,tempY,test_size=0.333)
print('TrainX size', TrainX.shape)
print('ValidX size', ValidX.shape)
print('TestX size', TestX.shape)
print('TrainY size', TrainY.shape)
print('ValidY size', ValidY.shape)
print('TestY size', TestY.shape)
sparse.save_npz('TrainInput_sparse_noleave.npz',TrainX)
sparse.save_npz('ValidInput_sparse_noleave.npz',ValidX)
sparse.save_npz('TestInput_sparse_noleave.npz',TestX)
sparse.save_npz('TrainOutput_sparse_noleave.npz',TrainY)
sparse.save_npz('ValidOutput_sparse_noleave.npz',ValidY)
sparse.save_npz('TestOutput_sparse_noleave.npz',TestY)
with open('AllSetSparseInOut_noleave.npz','wb') as handle:
    np.savez_compressed(handle,TrainX=TrainX,ValidX=ValidX,TestX=TestX,TrainY=TrainY,ValidY=ValidY,TestY=TestY)

TrainX size (2092295, 226)
ValidX size (1044580, 226)
TestX size (1045626, 226)
TrainY size (2092295, 1)
ValidY size (1044580, 1)
TestY size (1045626, 1)
